## Author: Chia-Yun (Sandy) Chiang

# Introduction

In this project, I'll use regularized model (Ridge & Lasso model) to predict the salary for sports player.

### Setup

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from sklearn.model_selection import KFold
import random
from numpy.random import randint
%matplotlib inline

In [2]:
plt.style.use('seaborn-white')
plt.style.use('seaborn-pastel')
plt.rcParams["figure.figsize"] = (24,8)

In [3]:
baseball_path = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQigI7-AKrL7SwQCp0zZjvvuaL12fpMxttsxSulXY4v2YjsvHWWP9MTo4fZqqCGXAFArICg0l38m9Re/pub?gid=1398581801&single=true&output=csv"

In [58]:
sport_df = pd.read_csv(baseball_path)

In [59]:
sport_df.head(2)

,Salary,count_at_bats,count_hits,count_home_runs,count_runs,RBI,count_walks,years_experience,cumulative_at_bats,cumulative_hits,cumulative_home_runs,cumulative_runs,cumulative_RBI,cumulative_walks,League,Division,PutOuts,Assists,Errors,NewLeague
0,875000,457,101,14,42,63,22,17,6521,1767,281,1003,977,619,A,W,389,39,4,A
1,1300000,441,118,28,84,86,68,8,2723,750,126,433,420,309,A,E,190,2,2,A


In [60]:
sport_df.tail(2)

,Salary,count_at_bats,count_hits,count_home_runs,count_runs,RBI,count_walks,years_experience,cumulative_at_bats,cumulative_hits,cumulative_home_runs,cumulative_runs,cumulative_RBI,cumulative_walks,League,Division,PutOuts,Assists,Errors,NewLeague
241,0,200,57,6,23,14,14,9,2516,684,46,371,230,195,N,W,69,1,1,N
242,0,346,98,5,31,53,30,16,5913,1615,235,784,901,560,A,E,0,0,0,A


In [61]:
sport_df.shape

(243, 20)

In [62]:
sport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Salary                243 non-null    int64 
 1   count_at_bats         243 non-null    int64 
 2   count_hits            243 non-null    int64 
 3   count_home_runs       243 non-null    int64 
 4   count_runs            243 non-null    int64 
 5   RBI                   243 non-null    int64 
 6   count_walks           243 non-null    int64 
 7   years_experience      243 non-null    int64 
 8   cumulative_at_bats    243 non-null    int64 
 9   cumulative_hits       243 non-null    int64 
 10  cumulative_home_runs  243 non-null    int64 
 11  cumulative_runs       243 non-null    int64 
 12  cumulative_RBI        243 non-null    int64 
 13  cumulative_walks      243 non-null    int64 
 14  League                243 non-null    object
 15  Division              243 non-null    ob

In [63]:
sport_df.isnull().sum()

Salary                  0
count_at_bats           0
count_hits              0
count_home_runs         0
count_runs              0
RBI                     0
count_walks             0
years_experience        0
cumulative_at_bats      0
cumulative_hits         0
cumulative_home_runs    0
cumulative_runs         0
cumulative_RBI          0
cumulative_walks        0
League                  0
Division                0
PutOuts                 0
Assists                 0
Errors                  0
NewLeague               0
dtype: int64

In [64]:
sport_df.describe().round()

,Salary,count_at_bats,count_hits,count_home_runs,count_runs,RBI,count_walks,years_experience,cumulative_at_bats,cumulative_hits,cumulative_home_runs,cumulative_runs,cumulative_RBI,cumulative_walks,PutOuts,Assists,Errors
count,243.0,243.0,243.0,243.0,243.0,243.0,243.0,243.0,243.0,243.0,243.0,243.0,243.0,243.0,243.0,243.0,243.0
mean,396486.0,378.0,100.0,11.0,50.0,47.0,38.0,7.0,2654.0,718.0,72.0,362.0,332.0,264.0,284.0,103.0,8.0
std,420524.0,151.0,46.0,8.0,25.0,26.0,21.0,5.0,2345.0,657.0,89.0,338.0,334.0,278.0,275.0,134.0,6.0
min,0.0,16.0,2.0,0.0,1.0,0.0,0.0,1.0,19.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
25%,70000.0,256.0,64.0,4.0,31.0,29.0,22.0,4.0,826.0,210.0,14.0,100.0,92.0,66.0,114.0,6.0,3.0
50%,250000.0,360.0,95.0,8.0,48.0,44.0,34.0,6.0,1931.0,510.0,39.0,258.0,230.0,174.0,211.0,33.0,6.0
75%,666500.0,511.0,136.0,16.0,68.0,64.0,52.0,11.0,3890.0,1038.0,93.0,524.0,424.0,330.0,322.0,152.0,10.0
max,1975000.0,687.0,238.0,40.0,117.0,121.0,105.0,24.0,14053.0,4256.0,548.0,2165.0,1659.0,1566.0,1378.0,492.0,32.0


- The minimum salary is 0 > need to check

In [65]:
obj_list = list(sport_df.select_dtypes(include = ["object"]).columns)
sport_df[obj_list].describe()

,League,Division,NewLeague
count,243,243,243
unique,2,2,2
top,A,W,A
freq,138,122,142


In [66]:
# Check the records which containing salary = 0 
sport_df[sport_df["Salary"]==0]

,Salary,count_at_bats,count_hits,count_home_runs,count_runs,RBI,count_walks,years_experience,cumulative_at_bats,cumulative_hits,cumulative_home_runs,cumulative_runs,cumulative_RBI,cumulative_walks,League,Division,PutOuts,Assists,Errors,NewLeague
184,0,397,114,23,67,67,53,13,5589,1632,241,906,926,716,A,E,244,2,4,A
185,0,542,140,12,46,75,41,16,7099,2130,235,987,1089,431,A,E,697,61,9,A
186,0,401,100,2,60,19,28,4,876,238,2,126,44,55,N,E,193,11,4,N
187,0,395,106,16,48,56,35,10,2303,571,86,266,323,248,A,E,709,41,7,A
188,0,278,69,3,24,21,29,8,2079,565,32,258,192,162,N,W,142,210,10,N
189,0,227,46,7,23,20,12,5,1325,324,44,156,158,67,A,W,92,2,2,A
190,0,517,141,27,70,87,52,9,3571,994,215,545,652,337,N,W,1378,102,8,N
191,0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,A
192,0,19,7,0,1,2,1,4,41,13,1,3,4,4,A,E,0,0,0,A
193,0,195,55,5,24,33,30,8,1313,338,25,144,149,153,N,E,83,2,1,N


- It seems like each rows do represent a real person with their sports records. Do not know why they have salary equals to 0.
- Since our objective is to predict the salary for sport players, these unusual rows might affect our prediction. As a result, I would like to assume these rows might be caused by some erros, and I would like to drop them at this time.

In [67]:
sport_df = sport_df[sport_df["Salary"]!=0]

In [68]:
sport_df.shape

(184, 20)

In [69]:
# create dummy for categorical column
sport_df = pd.get_dummies(data=sport_df, columns=["League", "Division","NewLeague"])

In [70]:
sport_df.head(3)

,Salary,count_at_bats,count_hits,count_home_runs,count_runs,RBI,count_walks,years_experience,cumulative_at_bats,cumulative_hits,cumulative_home_runs,cumulative_runs,cumulative_RBI,cumulative_walks,PutOuts,Assists,Errors,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
0,875000,457,101,14,42,63,22,17,6521,1767,281,1003,977,619,389,39,4,1,0,0,1,1,0
1,1300000,441,118,28,84,86,68,8,2723,750,126,433,420,309,190,2,2,1,0,1,0,1,0
2,940000,593,152,23,69,75,53,6,2765,686,133,369,384,321,315,10,6,1,0,0,1,1,0


In [71]:
# Prepare data 
y = sport_df["Salary"]
X = sport_df.drop(columns="Salary")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [72]:
# Ridge Model
cv_kfold = KFold(5, random_state=42, shuffle=True)
alpha = [i/1000.0 for i in range(1, 90, 1)]
ridgecv_model = RidgeCV(alphas= alpha, cv = cv_kfold, normalize=True)
ridgecv_model.fit(X_train, y_train)
y_pred = ridgecv_model.predict(X_test)
ridgecv_best_alpha = ridgecv_model.alpha_
ridgecv_coef = ridgecv_model.coef_
ridgecv_mse = mean_squared_error(y_test, y_pred)
ridgecv_training_score = ridgecv_model.score(X_train, y_train)
ridgecv_testing_score = ridgecv_model.score(X_test, y_test)
print("Ridge Model:")
print("Best alpha is {}".format(ridgecv_best_alpha))
print("MSE is {}".format(ridgecv_mse))
print("Training Score is {}".format(ridgecv_training_score) )
print("Testing Score is {}".format(ridgecv_testing_score) )




Ridge Model:
Best alpha is 0.089
MSE is 53794389463.221115
Training Score is 0.5357300305369941
Testing Score is 0.6027172826524301


In [73]:
# Show coefficient 
pd.DataFrame(ridgecv_coef, index = X.columns).sort_values(by=0)

,0
Division_W,-33354.076800
League_A,-32264.057207
NewLeague_N,-14936.239327
years_experience,-5698.748747
Errors,-5535.048568
count_at_bats,-473.313353
cumulative_walks,-188.504243
cumulative_at_bats,3.071578
Assists,29.661001
cumulative_RBI,85.850750


**Ridge Model Summary**
- After some trail and error, we found that for ridge model, the smaller alpha will lead to better model score. As a result, we used ridgecv model to help us find best alpha between 0.001 to 0.09.
- The best alpha for this model is 0.089. The model training score is 0.536; testing score is 0.603 with very high MSE score.
- This model performes bad for predicting salary.
- According to coefficient table above, we could find that ridge model penalize Division_W, League_A, NewLeague_N the most, which indicate these attritubes are less related to salary.
- We found that Division_E, League_N, NewLeague_A are the top three important attritubes. In other words, if sport players would like to have better salaries, consider these factors would be helpful.

In [74]:
# Lasso Model
cv_kfold = KFold(10, random_state=42, shuffle=True)
alpha = [i/10 for i in range(1000, 10000, 1)]
lasso_model = LassoCV(alphas= alpha , cv = cv_kfold, normalize=True, max_iter = 1000)
lasso_model.fit(X_train, y_train)
y_pred = lasso_model.predict(X_test)
lasso_best_alpha = lasso_model.alpha_
lasso_coef = lasso_model.coef_
lasso_mse = mean_squared_error(y_test, y_pred)
lasso_training_score = lasso_model.score(X_train, y_train)
lasso_testing_score = lasso_model.score(X_test, y_test)
print("Lasso Model:")
print("Best alpha is {}".format(lasso_best_alpha))
print("MSE is {}".format(lasso_mse))
print("Training Score is {}".format(lasso_training_score) )
print("Testing Score is {}".format(lasso_testing_score) )

Lasso Model:
Best alpha is 218.0
MSE is 65451358075.77486
Training Score is 0.5779415523926821
Testing Score is 0.5166281530491107


In [75]:
pd.DataFrame(lasso_coef, index = X.columns).sort_values(by=0).round(3)

,0
League_A,-38483.840
years_experience,-8326.488
Errors,-5130.186
count_at_bats,-1746.609
cumulative_walks,-572.233
cumulative_at_bats,-36.056
Division_W,-0.000
cumulative_RBI,0.000
NewLeague_A,0.000
NewLeague_N,-0.000


**Lasso Model Summary**
- After some trail and error, we found that for lasso model, the greater alpha will lead to better model score. As a result, we used lassocv model to help us find best alpha between 100 to 1000.
- The best alpha for this model is 218. The model training score is 0.5779; testing score is 0.5167 with very high MSE score.
- This model performes bad for predicting salary.
- According to coefficient table above, we could find that lasso model penalize League_A, years_experience, Errors the most, which indicate these attritubes are less related to salary.
- We found that Division_E, count_hits, count_home_runs are the top three important attritubes. In other words, if sport players would like to have better salaries, consider to join Division_E, and also focus on having more hits and home runs would be helpful.

**Overall Summary** 
- We could see that lasso model could have 0 as coefficient; while ridge model could not have 0 as coefficient.
- Both ridge model and lasso model perform bad in salary prediction.
- If we really need to predict sport players salary, I would use ridge model for prediction since it has higher testing score.

**Reference**
- [Harvard University - Lab 5: Regularization and Cross-Validation](https://harvard-iacs.github.io/2018-CS109A/labs/lab-5/solutions/)
- [How to Develop Ridge Regression Models in Python](https://machinelearningmastery.com/ridge-regression-with-python/)
